<h2>Introduction</h2>    

A travel agency is going offer its customers recommendation to the locations (neighborhoods) for given cuisines in major neighborhoods in Seattle, WA area. The App will allow users select from a list of cuisines then recommend the top 3 neighborhoods where they can have the most choices.

There are 91 neighborhoods in Seattle area. In this project, I implemented two features:

- The customers can query the restaurants for a specified cuisine in the major Seattle neighborhoods, which have 'Seattle' in name.
- The customers can query all restaurants (limit to 50 for demo purpose) in any one of Seattle neighborhood.

<h3>Import needed packages</h3>

In [1]:
import requests
import numpy as np
import pandas as pd
import random

In [122]:
!pip install geopy
import geopy
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


<h2>Data Section</h2>   

- Collecting the Latitude and Longitude for all neighborhoods in Seattle, WA area.
- Select the major neighborhood in Seatlle area.
- Using Foursquare location data API to extract cuisine venues by food categories that will be used for each neighborhoods in Seattle area. 
- Select top neighborhoods (by ordering the displayed neighborhoods) where there are most choices of the chosen cuisine to be select from.

In [3]:
import bs4
from bs4 import BeautifulSoup

In [4]:
wikipage_seattle = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle')

soup = BeautifulSoup(wikipage_seattle.content, 'lxml')
tbl = soup.find(class_="wikitable sortable")

df = pd.read_html(str(tbl))[0]
#neighborhoods = [s + ', WA' for s in df.iloc[:,1].reset_index(drop=True).tolist()]
neighborhoods = df.iloc[1:,1].reset_index(drop=True).tolist()

neighborhoods

['North Seattle',
 'Broadview',
 'Bitter Lake',
 'North Beach / Blue Ridge',
 'Crown Hill',
 'Greenwood',
 'Northgate',
 'Haller Lake',
 'Pinehurst',
 'North College Park (Licton Springs)',
 'Maple Leaf',
 'Lake City',
 'Cedar Park',
 'Matthews Beach',
 'Meadowbrook',
 'Olympic Hills',
 'Victory Heights',
 'Wedgwood',
 'View Ridge',
 'Sand Point',
 'Roosevelt',
 'Ravenna',
 'Bryant',
 'Windermere',
 'Hawthorne Hills',
 'Laurelhurst',
 'University District (U District)',
 'University Village',
 'Wallingford',
 'Northlake',
 'Green Lake',
 'Fremont',
 'Phinney Ridge',
 'Ballard',
 'West Woodland',
 'Whittier Heights',
 'Adams',
 'Sunset Hill',
 'Loyal Heights',
 'Central Seattle',
 'Magnolia',
 'Lawton Park',
 'Briarcliff',
 'Southeast Magnolia',
 'Interbay',
 'Queen Anne',
 'North Queen Anne',
 'East Queen Anne',
 'Lower Queen Anne',
 'West Queen Anne',
 'Capitol Hill',
 'Portage Bay[95] / Roanoke',
 'Broadway',
 'Pike-Pine Corridor / Pike/Pine[97][98][99]',
 'Montlake',
 'Stevens',
 'I

<h3>The Major neighborhoods (the name contains 'Seattle') in Seattle area.</h3>

In [5]:
seattle_neighborhoods = pd.DataFrame(columns=['Neighborhood', 'Latitude', 'Longitude'])

geolocator = Nominatim(user_agent="ny_explorer")

for i, address in enumerate(neighborhoods):
    location = geolocator.geocode(address + ', WA')
    if(pd.isnull(location) == False):
        seattle_neighborhoods.loc[i] = [address, location.latitude, location.longitude]

seattle_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,North Seattle,47.660773,-122.291497
1,Broadview,47.722320,-122.360407
2,Bitter Lake,47.726236,-122.348764
3,North Beach / Blue Ridge,47.700278,-122.396189
4,Crown Hill,47.694715,-122.371459


Use Foursquare APIs

In [121]:
CLIENT_ID = 'O1WQKL3EGFHCYD35S0Y4AAAZEBTSFF5KIUXSXEN5H2BIAZ3Y' # your Foursquare ID
CLIENT_SECRET = '0WZXE0XAFETCTVMLS14MAVKMTYDWXGWBGJF3TWGEAC4DTK0J' # your Foursquare Secret
VERSION = '20190425' # Foursquare API version

# these value can be changed accordingly
LIMIT = 50
radius = 500

### Get Categories name and Id with the Foursquare APIs
##### We only need Food catigories. Get relevant part of JSON and transform it into a pandas dataframe

In [8]:
result = requests.get("https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}".format(
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION)).json()
    
df_foodcatigories = pd.DataFrame(columns=['CategoryName','CategoryId'])
foodCategories = [r for r in result['response']['categories'] if r['name']=='Food'] 

for i, c in enumerate(foodCategories[0]['categories']):
    df_foodcatigories.loc[i] = [c['name'], c['id']]
    
df_foodcatigories.head()

,CategoryName,CategoryId
0,Afghan Restaurant,503288ae91d4c4b30a586d67
1,African Restaurant,4bf58dd8d48988d1c8941735
2,American Restaurant,4bf58dd8d48988d14e941735
3,Asian Restaurant,4bf58dd8d48988d142941735
4,Australian Restaurant,4bf58dd8d48988d169941735


In [9]:
df_foodcatigories.shape[0]

91

<h2>Explore restaurants in the Seattle neighborhood</h2>   
- Create a function to return the name, style, and address of selected cuisine restaurants in major Seattle neighborhoods.
- Create a function to return the first 50 restaurants with name, cuisine style, and address in a given Seattle neighborhood.

In [30]:
def CuisineRestaurantinArea(client_id, client_secret, version, lat, lon, radius, categoryId):
    url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&intent=browse&radius={}&categoryId={}".format(
        client_id, 
        client_secret,
        version,
        lat,
        lon,
        radius,
        categoryId)
    #print(url)
    foodvenues = requests.get(url).json()   
    #print(foodvenues)
    df = pd.DataFrame(columns = ['Name', 'Style', 'Address'])
    idx = 0
    if len(foodvenues['response']['venues']) > 0:     
        for v in foodvenues['response']['venues']:
            address = v['location']['formattedAddress']
            df.loc[idx] = [v['name'], v['categories'][0]['name'], address]
            idx += 1
    return df

#### Get the 'major' Seattle neighborhoods that name contain word Seattle

In [12]:
df_seattle = seattle_neighborhoods[seattle_neighborhoods['Neighborhood'].apply(lambda tag: 'Seattle' in tag)].reset_index(drop=True)

df_seattle

,Neighborhood,Latitude,Longitude
0,North Seattle,47.660773,-122.291497
1,Central Seattle,47.612694,-122.303205
2,"South Lake Union, Seattle",47.623161,-122.338382
3,"Cascade, Seattle",47.620241,-122.333318
4,West Seattle,47.570932,-122.386517


<h3>Find the restaurants of selected cuisine in the Seattle neighborhoods</h3>      

Query with Foursqaure API and return restaurants for a specified cuisine in the neighborhoods, which have 'Seattle' in name.

In [70]:
def GetCuisineInformation(cuisine_category):      
    c_id = cuisine_category['CategoryId'].values[0]
    seattle_cuisines_dict = {}
    restaurant_count = pd.DataFrame(columns=['Name', 'Count'])
    for i, n in df_seattle.iterrows():
        cuisines = CuisineRestaurantinArea(CLIENT_ID, CLIENT_SECRET, VERSION, n['Latitude'], n['Longitude'], radius, c_id)
        if cuisines.shape[0] == 0:
            continue

        seattle_cuisines_dict[n['Neighborhood']] = cuisines
        restaurant_count.loc[i] = [n['Neighborhood'], cuisines.shape[0]]
        
    restaurant_count.sort_values(by='Count', inplace=True, ascending=False)
    for i, c in restaurant_count.iterrows():
        df = seattle_cuisines_dict[c['Name']]
        yield (c['Name'], df)

#### Display cuisine restaurants for the Seattle neighborhoods (with 'Seattle' in name) with the neighborhood sorted by number of the cuisine found ascendingly.    

In [54]:
# looping to allow user select cuisine. if 'Q' is enter, stop
while True:
    cuisine_prefix = input('Cuisine(i.e. Asian or Italian) or Q to stop:')
    if cuisine_prefix == 'Q':
        break
    else:
        cuisine_category = df_foodcatigories[df_foodcatigories['CategoryName'].apply(lambda tag: tag.startswith(cuisine_prefix))]
        if cuisine_category.shape[0] == 0:
            print("Not find cuisine:{} please try again!".format(cuisine_prefix))
            continue
            
        for (n, df) in GetCuisineInformation(cuisine_category):
            print('\033[1m' + n  + '\033[0m')
            display(df)
            print('')

Cuisine(i.e. Asian or Italian) or Q to stop:Asian
Cascade, Seattle


,Name,Style,Address
0,Ba Bar SLU,Vietnamese Restaurant,"[500 Terry Ave N (Republican), Seattle, WA 981..."
1,Phở Bac,Vietnamese Restaurant,"[1809 Minor Ave (at 8th Ave.), Seattle, WA 981..."
2,Kigo Kitchen,Asian Restaurant,"[1730 Minor Ave, Seattle, WA 98101, United Sta..."
3,Teinei,Japanese Restaurant,[1256 Republican St (Pontius Ave N and Yale Av...
4,Vinason Pho And Grill,Vietnamese Restaurant,"[2134 Westlake Ave (9th Ave), Seattle, WA 9812..."
5,Thai Thani Kitchen,Thai Restaurant,"[201 Boren Ave N (John), Seattle, WA 98109, Un..."
6,Mio Sushi,Sushi Restaurant,"[120 Westlake Ave N, Seattle, WA 98109, United..."
7,Mala & Satay,Vietnamese Restaurant,"[Seattle, WA 98109, United States]"
8,Midori Teriyaki,Japanese Restaurant,"[1120 Howell St (at Minor), Seattle, WA 98101,..."
9,ON THE FLY,Asian Restaurant,"[950 Thomas Street, Seattle, WA 98119, United ..."



South Lake Union, Seattle


,Name,Style,Address
0,Ba Bar SLU,Vietnamese Restaurant,"[500 Terry Ave N (Republican), Seattle, WA 981..."
1,Hurry Curry of Tokyo,Japanese Curry Restaurant,"[825 Harrison St (8th Ave N), Seattle, WA 9810..."
2,Revel SLU,Korean Restaurant,"[513 Westlake Ave N, Seattle, WA 98109, United..."
3,Thai Thani Kitchen,Thai Restaurant,"[201 Boren Ave N (John), Seattle, WA 98109, Un..."
4,Mio Sushi,Sushi Restaurant,"[120 Westlake Ave N, Seattle, WA 98109, United..."
5,Sizzle & Crunch,Vietnamese Restaurant,"[500 9th Ave N Ste 150, Seattle, WA 98109, Uni..."
6,Mala & Satay,Vietnamese Restaurant,"[Seattle, WA 98109, United States]"
7,ON THE FLY,Asian Restaurant,"[950 Thomas Street, Seattle, WA 98119, United ..."
8,Kigo Kitchen,Asian Restaurant,"[210 Westlake Ave N, Seattle, WA 98109, United..."
9,Yummy Box,Food Truck,"[Washington, United States]"



North Seattle


,Name,Style,Address
0,Marlai Thai Cuisine,Thai Restaurant,"[3719 NE 45th St, Seattle, WA 98105, United St..."
1,Toshi's Teriyaki The Original,Japanese Restaurant,"[3715 NE 45th St, Seattle, WA 98105, United St..."
2,Sunrice,Asian Restaurant,"[3513 NE 45th St, Seattle, WA 98105, United St..."
3,China Village,Chinese Restaurant,"[3224 NE 45th St, Seattle, WA 98105, United St..."



West Seattle


,Name,Style,Address
0,Srivilai,Thai Restaurant,"[3247 Califronia SW (SW Hinds), Seattle, WA 98..."



Cuisine(i.e. Asian or Italian) or Q to stop:Q


We can find all of the seattle neighborhoods in dataframe: seattle_neighborhoods

In [94]:
seattle_neighborhoods.head(6)

,Neighborhood,Latitude,Longitude
0,North Seattle,47.660773,-122.291497
1,Broadview,47.722320,-122.360407
2,Bitter Lake,47.726236,-122.348764
3,North Beach / Blue Ridge,47.700278,-122.396189
4,Crown Hill,47.694715,-122.371459
5,Greenwood,47.690981,-122.354877


<h3>Find the first 50 Restaurants in a given Seattle Neighborhood</h3>   
This can be scaled up to return more or less by change limit vlaue.

<h4>Explore limited (50) number of all kind restaurants in the Seattle neighborhood</h4>   

Create a function to return the name, cuisine, and address of the restaurants in that neighborhood.

In [105]:
def GetAllRestaurantinNeighborhood(client_id, client_secret, version, lat, lon, radius, limit, c_id):
    url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&intent=browse&radius={}&limit={}&categoryId={}".format(
        client_id, 
        client_secret,
        version,
        lat,
        lon,
        radius,
        limit,
        c_id)
    foodvenues = requests.get(url).json()
    df = pd.DataFrame(columns = ['Name', 'Style', 'Address'])
    idx = 0
    if len(foodvenues['response']['venues']) > 0:     
        for v in foodvenues['response']['venues']:
            address = v['location']['formattedAddress']
            df.loc[idx] = [v['name'], v['categories'][0]['name'], address]
            idx += 1
    return df

In [120]:
def GetNeighborhoodRestaurants(categoryId, neighborhoodName):
    n = seattle_neighborhoods[seattle_neighborhoods["Neighborhood"] == neighborhoodName].reset_index(drop=True)

    df = GetAllRestaurantinNeighborhood(CLIENT_ID, CLIENT_SECRET, VERSION, n.loc[0]['Latitude'], n.loc[0]['Longitude'], radius, LIMIT,categoryId)
    return df

####  Display first 50 (all kind of cuisines) restaurants for the selected Seattle neighborhood.    
Looping to allow user select the neighborhoods repeatly, till enter 'Q' that end the loop.

In [119]:
# looping to allow user select seattle neighborhoods. if 'Q' is enter, stop
while True:
    neighborhood = input('Seattle Neiborhood Name or Q to stop:')
    if neighborhood == 'Q':
        break
    else:
        n_names = seattle_neighborhoods[seattle_neighborhoods['Neighborhood'].apply(lambda tag: tag == neighborhood)]
        if n_names.shape[0] == 0:
            print("Not find Neighborhood: {} please try again!".format(neighborhood))
            continue
        # food category id    
        food_categoryId = '4d4b7105d754a06374d81259'    
        df = GetNeighborhoodRestaurants(food_categoryId,neighborhood)
        print('\033[1m' + neighborhood  + '\033[0m')
        display(df)
        print('')


Seattle Neiborhood Name or Q to stop:reenwood
Not find Neighborhood: reenwood please try again!
Seattle Neiborhood Name or Q to stop:Greenwood
  Neighborhood   Latitude   Longitude
0    Greenwood  47.690981 -122.354877
Greenwood


,Name,Style,Address
0,North Star Diner,Diner,"[8700 Greenwood Ave N (N 87th Street), Seattle..."
1,Coyle's Bakeshop,Bakery,"[8300 Greenwood Ave N (N 83rd St), Seattle, WA..."
2,FlintCreek Cattle Co.,Steakhouse,"[8421 Greenwood Ave N (85TH ST), Seattle, WA 9..."
3,The Lodge,Gastropub,"[8501 Greenwood Ave N (N 85th Street), Seattle..."
4,Fred Meyer Breakfast Bar,Breakfast Spot,"[85th Street, Seattle, WA, United States]"
5,Gorditos,Mexican Restaurant,"[213 N 85th St, Seattle, WA 98103, United States]"
6,Chaco Canyon Organic Café,Vegetarian / Vegan Restaurant,"[8404 Greenwood Ave N (85th), Seattle, WA 9810..."
7,Chef King,Chinese Restaurant,"[Seattle, WA 98103, United States]"
8,Valhalla Sandwiches,Sandwich Place,"[8202 Greenwood Ave N (N. 82nd), Seattle, WA 9..."
9,Monkey Grind Espresso Bar,Coffee Shop,"[518 N 85th St (at Evanston), Seattle, WA 9810..."



Seattle Neiborhood Name or Q to stop:Broadview
  Neighborhood  Latitude   Longitude
0    Broadview  47.72232 -122.360407
Broadview


,Name,Style,Address
0,Ichi Roll Teriyaki,Sushi Restaurant,"[326 N 125th St, Seattle, WA 98133, United Sta..."
1,Domino's Pizza,Pizza Place,"[302 N 125th St, Seattle, WA 98133, United Sta..."
2,Chada Thai,Thai Restaurant,"[308 N 125th St, Seattle, WA 98133, United Sta..."
3,Brother John's Pizzeria,Food,"[12427 Greenwood Ave N, Seattle, WA 98133, Uni..."



Seattle Neiborhood Name or Q to stop:Bitter Lake
  Neighborhood   Latitude   Longitude
0  Bitter Lake  47.726236 -122.348764
Bitter Lake


,Name,Style,Address
0,Chick-fil-A,Fast Food Restaurant,"[12801 Aurora Ave N (N 128th St), Seattle, WA ..."
1,Green Leaf Vietnamese Restaurant,Vietnamese Restaurant,"[13200 Aurora Ave N, Seattle, WA 98133, United..."
2,HK Dim Sum,Dim Sum Restaurant,"[13200 Aurora Ave N Apt F, Seattle, WA 98133, ..."
3,Starbucks,Coffee Shop,"[13025 Aurora Ave N, Seattle, WA 98133, United..."
4,Rain Cafe,Café,"[13200 Aurora Ave N, Suite C, Seattle, WA 9813..."
5,Qian Noodles,Noodle House,"[13510 Aurora Ave N, Seattle, WA 98133, United..."
6,Pop Pop Thai Street Food,Thai Restaurant,"[13242 Aurora Ave N, Seattle, WA 98133, United..."
7,Ivar's,Seafood Restaurant,"[13448 Aurora Ave N (at N 135th St.), Seattle,..."
8,kiki bakery,Bakery,"[13200 Aurora Ave N Apt E (130th), Seattle, WA..."
9,Burger King,Fast Food Restaurant,"[13241 Aurora Ave N, Seattle, WA 98133, United..."



Seattle Neiborhood Name or Q to stop:Q


<h2>Conclusion</h2>   

- This application let the customers chose the cuisine information of then return the cuisine restaurants in the major Seattle neighhoods, which have 'Seattle' in name, grouping by the neighborhood that ordered by the most cuisine restaurants found. It can be easily scaled to all neighborhoods in Seattle or other areas.   

- The application let the customers chose a Seattle neighborhood then return the information of the first 50 restaurants found in that neighborhood. It can be easily scaled to return all restaurants in the chosen neighborhood.